# Homework #2



## Question #1

There are four text files in the data folder

* Atatürk's "Nutuk" in Turkish
* Dicken's novel "Great Expectations" in English
* Flauberts' novel "Madam Bovary" in French
* A text file `unknown.txt` in an unknown language

Your tasks are

* Calculate how many times each character (letter) appear in each text.
* Calculate the character distributions, i.e. using the character counts, calculate the probability of each character appearing in the text.
* Find the set of characters common to all three texts.
* Using the common set and the KL-divergence, show that each language have different character distributions.
* Determine the language of the text file `unknown.txt` KL-divergence measure.

In [155]:
import pandas as pd
import numpy as np
import string

nutuk=open('ataturk_nutuk.txt','r+',encoding='utf-8')
dickens=open("dickens_great_expectations.txt","r+",encoding="utf-8")
bovary= open("flaubert_madame_bovary.txt","r+",encoding='utf-8') 
unknown=open("unknown.txt","r+",encoding='utf-8')

counter1=dict()
counter2=dict()
counter3=dict()
counter4=dict()

text1=nutuk.read()
text2=dickens.read()
text3=bovary.read()
text4=unknown.read()

words1=text1.split()
words2=text2.split()
words3=text3.split()
words4=text4.split()

for i in range(len(words1)):    
    words1[i]=words1[i].lower()
    words1[i] = words1[i].translate(words1[i].maketrans("", "", string.punctuation)) 
for i in range(len(words2)):    
    words2[i]=words2[i].lower()
    words2[i] = words2[i].translate(words2[i].maketrans("", "", string.punctuation)) 
for i in range(len(words3)):    
    words3[i]=words3[i].lower()
    words3[i] = words3[i].translate(words3[i].maketrans("", "", string.punctuation)) 
for i in range(len(words4)):    
    words4[i]=words4[i].lower()
    words4[i] = words4[i].translate(words4[i].maketrans("", "", string.punctuation)) 
print("letter amount of nutuk")

for word in words1:
    for char in word:
        if char in counter1:
            counter1[char]+=1
        else:
            counter1[char]=1
print(counter1)
print("letter amount of dickens")
for word in words2:
    for char in word:
        if char in counter2:
            counter2[char]+=1
        else:
            counter2[char]=1
print(counter2)
print("letter amount of bovary")
for word in words3:
    for char in word:
        if char in counter3:
            counter3[char]+=1
        else:
            counter3[char]=1
print(counter3)
print("letter amount of unknown")
for word in words4:
    for char in word:
        if char in counter4:
            counter4[char]+=1
        else:
            counter4[char]=1
print(counter4)


letter amount of nutuk
{'y': 29496, 'u': 39565, 'r': 74152, 'd': 45861, 'm': 70652, 'z': 15593, 'n': 78256, 'p': 7371, 'a': 118982, 'ç': 8613, 'l': 75010, 'ı': 44210, 'i': 100889, 'ş': 19525, 'g': 14552, 'e': 113678, 'ğ': 10530, 'ü': 19746, 'b': 29389, 'k': 48264, 't': 42730, 'h': 12834, 'ö': 7358, 'o': 20290, 's': 30794, '̇': 6346, 'â': 2643, 'v': 15618, 'c': 10929, 'î': 1469, 'f': 7489, '1': 14901, '5': 274, '2': 1432, '0': 13567, '9': 1132, '7': 184, '3': 478, '6': 177, 'j': 163, 'á': 2, '8': 209, 'ì': 5, 'û': 836, '4': 251, 'ó': 1, '\x1b': 25980, 'w': 66, 'í': 4, 'ú': 1, '·': 6, '\x7f': 22, 'q': 1, 'é': 30, 'x': 3, 'ß': 1, '«': 6, '»': 6}
letter amount of dickens
{'\ufeff': 1, 't': 70355, 'h': 48738, 'e': 93720, 'p': 13417, 'r': 42122, 'o': 61277, 'j': 1759, 'c': 17625, 'g': 16871, 'u': 22257, 'n': 53781, 'b': 12451, 'k': 7680, 'f': 16130, 'a': 64021, 'x': 1126, 'i': 55801, 's': 45936, 'y': 16410, 'l': 28564, 'd': 37182, 'w': 20526, 'm': 22906, 'v': 6896, '1': 64, '9': 11, '8': 12,

In [152]:
lc1=list(counter1.values())
lc2=list(counter2.values())
lc3=list(counter3.values())
lc4=list(counter4.values())

lc1_k=list(counter1.keys())
lc2_k=list(counter2.keys())
lc3_k=list(counter3.keys())
lc4_k=list(counter4.keys())

total_letter_c1=np.sum(lc1)
total_letter_c2=np.sum(lc2)
total_letter_c3=np.sum(lc3)
total_letter_c4=np.sum(lc4)

probsc1=lc1/total_letter_c1
probsc2=lc2/total_letter_c2
probsc3=lc3/total_letter_c3
probsc4=lc4/total_letter_c4

zip1=zip(lc1_k,probsc1)
zip2=zip(lc2_k,probsc2)
zip3=zip(lc3_k,probsc3)
zip4=zip(lc4_k,probsc4)

dict1=dict(zip1)
dict2=dict(zip2)
dict3=dict(zip3)
dict4=dict(zip4)


In [154]:
common=set( dict1.keys() ) & set( dict2.keys() ) & set(dict3.keys()) & set(dict4.keys())  
common

{'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y'}

In [193]:
from scipy.stats import entropy

commondict1=dict()
commondict2=dict()
commondict3=dict()
commondict4=dict()
for letter in common:
    commondict1[letter]=dict1[letter]
    commondict2[letter]=dict2[letter]
    commondict3[letter]=dict3[letter]
    commondict4[letter]=dict4[letter]
    
valuecommon1=list(commondict1.values())
valuecommon2=list(commondict2.values())
valuecommon3=list(commondict3.values())
valuecommon4=list(commondict4.values())

In [200]:
print("entropy between 4 and 1 is {}".format(entropy(valuecommon4,valuecommon1)))
print("entropy between 4 and 2 is {}".format(entropy(valuecommon4,valuecommon2)))
print("entropy between 4 and 3 is {}".format(entropy(valuecommon4,valuecommon3)))
print("min entropy is between 2 and 4, so it's language is english.")

entropy between 4 and 1 is 0.43005894096312886
entropy between 4 and 2 is 0.038013041187527796
entropy between 4 and 3 is 0.23744692641349954
min entropy is between 2 and 4, so it's language is english.


## Question #2

For this question consider the [Car Evaluation Data Set](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation) from UCI. Here is the [direct link](https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data) to the dataset.

Make [contingency tables](https://en.wikipedia.org/wiki/Contingency_table#:~:text=In%20statistics%2C%20a%20contingency%20table,%2C%20engineering%2C%20and%20scientific%20research.) of the columns (using [`crosstab`](https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html) function from [pandas](https://pandas.pydata.org)) and figure out which pairs of columns are dependent and independent. Explain your result using statistical tests.

In [259]:
from scipy.stats import chisquare

data=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data",header=None)

a=pd.crosstab(data[0],data[1])
b=pd.crosstab(data[1],data[2])
c=pd.crosstab(data[2],data[3])
d=pd.crosstab(data[3],data[4])
e=pd.crosstab(data[4],data[5])
f=pd.crosstab(data[5],data[6])
g=pd.crosstab(data[4],data[6])
h=pd.crosstab(data[3],data[6])
i=pd.crosstab(data[2],data[6])
j=pd.crosstab(data[1],data[6])
k=pd.crosstab(data[0],data[6])

print(chisquare(a,axis=None))
print(chisquare(b,axis=None))
print(chisquare(c,axis=None))
print(chisquare(d,axis=None))
print(chisquare(e,axis=None))
print(chisquare(f,axis=None))
print(chisquare(g,axis=None))
print(chisquare(h,axis=None))
print(chisquare(i,axis=None))
print(chisquare(j,axis=None))
print(chisquare(k,axis=None))
print("6th column is dependent with each columns...")

Power_divergenceResult(statistic=0.0, pvalue=1.0)
Power_divergenceResult(statistic=0.0, pvalue=1.0)
Power_divergenceResult(statistic=0.0, pvalue=1.0)
Power_divergenceResult(statistic=0.0, pvalue=1.0)
Power_divergenceResult(statistic=0.0, pvalue=1.0)
Power_divergenceResult(statistic=2554.0555555555557, pvalue=0.0)
Power_divergenceResult(statistic=2059.4444444444443, pvalue=0.0)
Power_divergenceResult(statistic=2515.847222222222, pvalue=0.0)
Power_divergenceResult(statistic=2034.0, pvalue=0.0)
Power_divergenceResult(statistic=2104.4814814814813, pvalue=0.0)
Power_divergenceResult(statistic=2122.0, pvalue=0.0)
6th column is dependent with each columns...


## Question #3

For this question, use [Default of Credit Card Clients Data Set]() from UCI. Here is the [direct link](https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls) to the dataset.

Your tasks are

* Inspect the dataset.
* Would it be appropriate to form a linear regression model to predict the `default payment next month` variable? Explain.
* Form a [contingency table](https://en.wikipedia.org/wiki/Contingency_table#:~:text=In%20statistics%2C%20a%20contingency%20table,%2C%20engineering%2C%20and%20scientific%20research.) of the columns `SEX` vs `default payment next month` and `EDUCATION` vs `default payment next month`.
* Are there statistically verifiable relationships between credit card defaults, the gender of and the education level the borrower? Which is stronger? Quantify your analysis using [Chi Square Test](https://en.wikipedia.org/wiki/Chi-squared_test).

In [19]:
import pandas as pd
data=pd.read_excel("https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls")
print(" The target column consits of binary numbers, so this is not a linear regression problem.  ")


 The target column consits of binary numbers, so this is not a linear regression problem.  


In [28]:
X=data.drop(0,axis=0)
y=data["Y"]

X=X.drop("Y",axis=1)


In [31]:
y=y.drop(0,axis=0)

In [37]:
data

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29997,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29998,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29999,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [38]:
a=pd.crosstab(X["X2"],y)
b=pd.crosstab(X["X3"],y)

In [40]:
print("sex vs payment")
a

Y,0,1
X2,,
1,9015,2873
2,14349,3763


In [41]:
print("education vs payment")
b

Y,0,1
X3,,
0,14,0
1,8549,2036
2,10700,3330
3,3680,1237
4,116,7
5,262,18
6,43,8


In [42]:
from scipy.stats import chisquare
print(chisquare(a,axis=None))
print(chisquare(b,axis=None))

Power_divergenceResult(statistic=11277.109866666666, pvalue=0.0)
Power_divergenceResult(statistic=71717.81573333332, pvalue=0.0)


#### P values are 0 so these features has a relationship between default payment and the test value of education is higher, so that means education is more related to payment

## Question #4

For this question, use the [Iris Dataset](https://archive.ics.uci.edu/ml/datasets/iris) from UCI.  Here is the [direct link](https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data) to the dataset.

Your tasks are

* Form a [K-NN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) model for this dataset.
* Test your model on random samples of your data and calculate its accuracy.
* Repeat your calculation 100 times and give an interval of accuracy values leaving the best 2.5% and worst 2.5% accuracy values.
* Is there a better way of doing this without repeating the calculation 100 times? Explain.
* Find the best parameter $k$ for your dataset for the K-NN model.

In [50]:
data=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",header=None)

In [97]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit

X=data.iloc[:,0:4]
y=data[4]
scores=[]

for i in range(100):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

    
    model2.fit(X_train,y_train)
    y_preds=model2.predict(X_test)
    sc=accuracy_score(y_test,y_preds)
    
    scores.append(sc)

In [98]:
print("Cross validation can be used instead of for loop...")

model3=KNeighborsClassifier(n_neighbors=3)
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cross_val_score(model3, X, y, cv=cv)

Cross validation can be used instead of for loop...


array([0.97777778, 0.93333333, 0.97777778, 0.91111111, 0.97777778])

In [111]:
ks=[1,2,3,5,10,25,50,100]
scores=[]
for k in ks:
    model4=KNeighborsClassifier(n_neighbors=k)
    cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
    a=cross_val_score(model4, X, y, cv=cv)
    meana=a.mean()
    scores.append(meana)


In [112]:
scores

[0.9644444444444444,
 0.9511111111111111,
 0.9555555555555555,
 0.96,
 0.96,
 0.9466666666666667,
 0.888888888888889,
 0.4666666666666667]

In [117]:
import numpy as np
print("k ={} is the best k-nn".format(ks[np.argmax(scores)]))

k =1 is the best k-nn


## Question #5

For this question, we are going to use [Concrete Slump Test Dataset](https://archive.ics.uci.edu/ml/datasets/Concrete+Slump+Test) from UCI. Here is the [direct link](https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/slump/slump_test.data) to the dataset.

Your tasks are

* Form three separate linear regression model for the following dependent variables:

  - SLUMP (cm)
  - FLOW (cm)
  - 28-day Compressive Strength (Mpa)
  
* Compare how well these models fit.

In [196]:
data=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/slump/slump_test.data")
data

,No,Cement,Slag,Fly ash,Water,SP,Coarse Aggr.,Fine Aggr.,SLUMP(cm),FLOW(cm),Compressive Strength (28-day)(Mpa)
0,1,273.0,82.0,105.0,210.0,9.0,904.0,680.0,23.0,62.0,34.99
1,2,163.0,149.0,191.0,180.0,12.0,843.0,746.0,0.0,20.0,41.14
2,3,162.0,148.0,191.0,179.0,16.0,840.0,743.0,1.0,20.0,41.81
3,4,162.0,148.0,190.0,179.0,19.0,838.0,741.0,3.0,21.5,42.08
4,5,154.0,112.0,144.0,220.0,10.0,923.0,658.0,20.0,64.0,26.82
...,...,...,...,...,...,...,...,...,...,...,...
98,99,248.3,101.0,239.1,168.9,7.7,954.2,640.6,0.0,20.0,49.97
99,100,248.0,101.0,239.9,169.1,7.7,949.9,644.1,2.0,20.0,50.23
100,101,258.8,88.0,239.6,175.3,7.6,938.9,646.0,0.0,20.0,50.50
101,102,297.1,40.9,239.9,194.0,7.5,908.9,651.8,27.5,67.0,49.17


In [210]:
model1=LinearRegression()
model2=LinearRegression()
model3=LinearRegression()
data=data.dropna()

X0=data.iloc[:,10]
X1=data.iloc[:,8]
X2=data.iloc[:,9]


In [219]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

model11=LinearRegression()
model22=LinearRegression()
model33=LinearRegression()
data=data.dropna()

y=np.array(data.iloc[:,10]).reshape(len(data),1)
X1=np.array(data.iloc[:,8]).reshape(len(data),1)
X2=np.array(data.iloc[:,9]).reshape(len(data),1)

X1_train,X1_test,y1_train,y1_test=train_test_split(X1,y,test_size=0.3)
model11.fit(X1_train,y1_train)
score11=model11.score(X1_test,y1_test)

X2_train,X2_test,y2_train,y2_test=train_test_split(X2,y,test_size=0.3)
model22.fit(X2_train,y2_train)
score2=model22.score(X2_test,y2_test)

X3_train,X3_test,y3_train,y3_test=train_test_split(X1,X2,test_size=0.3)
model33.fit(X3_train,y3_train)
score3=model33.score(X3_test,y3_test)

print(score1)
print(score2)
print(score3)

0.03001402841743439
-0.03842121374001173
0.8270377875516006


In [220]:
print("R^2 is better when it is close to 1, so model 3 fits pretty well. (Slump vs Flow) acc is {}".format(score3))

R^2 is better when it is close to 1, so model 3 fits pretty well. (Slump vs Flow) acc is 0.8270377875516006
